# WEB - SCRAPING

# 1. Librerías

In [1]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

import re

import pandas as pd
import time
from datetime import datetime as date

# 2. Configuración del driver

In [2]:
# Opciones de navegación
options =  webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

In [3]:
chrome_service = Service('driver_chrome\chromedriver.exe')

driver = webdriver.Chrome(service=chrome_service, options=options)

# 3. Iniciar el driver

In [4]:
url = 'https://www.cazoo.co.uk/cars/'


# Inicializamos el navegador
driver.get(url)


# Aceptar cookies www.cazoo.co.uk/cars/

WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                                      'button.cui__sc-mqi3p7-0 cHWLkA'.replace(' ', '.')))).click()

time.sleep(3)

# 4. Extracción de datos

In [6]:
def Extraer_Datos(fabricante, desc, precio, datos, ciudad, num_cars_page):
    
    for n in range(2, num_cars_page + 1):
        
        xpath_fabricante = '/html/body/div[1]/main/section/ol/li[' + str(n) + ']/div/div[1]/div[2]/div/div/p[1]'
        fabricante = Extraer_datos_xpath(xpath_fabricante, fabricante)
        
        xpath_desc = '/html/body/div[1]/main/section/ol/li[' + str(n) + ']/div/div[1]/div[2]/div/div/p[2]'
        desc = Extraer_datos_xpath(xpath_desc, desc)
        
        xpath_precio = '/html/body/div[1]/main/section/ol/li[' + str(n) + ']/div/div[2]/p'
        precio = Extraer_datos_xpath(xpath_precio, precio)
        
        xpath_datos = '/html/body/div[1]/main/section/ol/li[' + str(n) + ']/div/div[1]/div[2]/ul'
        datos = Extraer_datos_xpath(xpath_datos, datos)
        
        xpath_ciudad = '/html/body/div[1]/main/section/ol/li[' + str(n) + ']/div/div[3]/p/span'
        ciudad = Extraer_datos_xpath(xpath_ciudad, ciudad)
        
    return fabricante, desc, precio, datos, ciudad

In [7]:
def Extraer_datos_xpath(xpath, lista_datos):
    
    try:
        datos = driver.find_element(By.XPATH, xpath)
        lista_datos.append(datos.text)
    except (NoSuchElementException, StaleElementReferenceException):
        lista_datos.append('NA')
        
    return lista_datos

In [8]:
# Revisar número de coches en página 1

cars_page = driver.find_element(By.XPATH, '/html/body/div[1]/main/section/div[3]/div/div[1]/p')


first_num = int(cars_page.text.split('- ')[0].replace(' ',''))
last_num = int(cars_page.text.split('- ')[1].split(' ')[0])

num_cars_page = (last_num - first_num) + 2
num_cars_page

48

In [9]:
# Extraer datos página 1

fabricante = []
desc = []
precio = []
datos = []
ciudad = []


    
    
p1_fabricante, p1_desc, p1_precio, p1_datos, p1_ciudad = Extraer_Datos(fabricante, desc, precio, datos, ciudad, num_cars_page)


fabricante = p1_fabricante
desc = p1_desc
precio = p1_precio
datos = p1_datos
ciudad = p1_ciudad

In [10]:
# Revisar información última página

ultima_pagina = driver.find_element(By.XPATH, '/html/body/div[1]/main/section/div[4]')
texto_ultima_pagina = ultima_pagina.text

last_page = int(texto_ultima_pagina.split('\n')[-2])
# last_page

In [11]:
# Extraer datos pág. 2 hasta última página

pagina_actual = 2


# Hacer clic last_page-1 veces

while last_page > 1:
    
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                                'svg.cui__sc-t0exld-0 kgGzag'.replace(' ', '.')))).click()

    time.sleep(1)
    
    
    print('Obteniendo información página {}'.format(pagina_actual))
    
    # Revisar número de coches en la página
    
    cars_page = driver.find_element(By.XPATH, '/html/body/div[1]/main/section/div[2]/div/div[1]/p')
    first_num = int(cars_page.text.split('- ')[0].replace(' ',''))
    last_num = int(cars_page.text.split('- ')[1].split(' ')[0])

    
    num_cars_page = (last_num - first_num) + 2
    num_cars_page




    p_fabricante, p_desc, p_precio, p_datos, p_ciudad = Extraer_Datos(fabricante, desc, precio, datos, ciudad, num_cars_page)


    fabricante = p_fabricante
    desc = p_desc
    precio = p_precio
    datos = p_datos
    ciudad = p_ciudad
    
    
    
                                         
    last_page -= 1
          
    pagina_actual += 1 

Obteniendo información página 2
Obteniendo información página 3
Obteniendo información página 4
Obteniendo información página 5
Obteniendo información página 6
Obteniendo información página 7
Obteniendo información página 8
Obteniendo información página 9
Obteniendo información página 10
Obteniendo información página 11
Obteniendo información página 12
Obteniendo información página 13
Obteniendo información página 14
Obteniendo información página 15
Obteniendo información página 16
Obteniendo información página 17
Obteniendo información página 18
Obteniendo información página 19
Obteniendo información página 20
Obteniendo información página 21
Obteniendo información página 22
Obteniendo información página 23
Obteniendo información página 24
Obteniendo información página 25
Obteniendo información página 26
Obteniendo información página 27
Obteniendo información página 28
Obteniendo información página 29
Obteniendo información página 30
Obteniendo información página 31
Obteniendo informa

# 5. Almacenamiento de los datos

In [12]:
fecha_actual = date.today().strftime('%d') + '-' + date.today().strftime('%m') + '-' + date.today().strftime('%y')
horas_min_seg = date.today().strftime('%H') + 'h' + date.today().strftime('%M') + 'm' + date.today().strftime('%S') + 's'
# fecha_actual
# horas_min_seg

In [13]:
df_cars = pd.DataFrame({'Fabricante': fabricante, 'Descripción': desc, 'Datos': datos, 'Precio_£': precio, 'Ciudad': ciudad})

df_cars.to_csv('Datos_web_scraping/' + 'df_cars' + '_' + fecha_actual + '_' + horas_min_seg + '.csv', index=False)

driver.quit()

In [14]:
df_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9342 entries, 0 to 9341
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Fabricante   9342 non-null   object
 1   Descripción  9342 non-null   object
 2   Datos        9342 non-null   object
 3   Precio_£     9342 non-null   object
 4   Ciudad       9342 non-null   object
dtypes: object(5)
memory usage: 365.1+ KB
